In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sys
sys.path.insert(0, '/home/atbin/workd/imputation/package/src/imputation/configs/')
sys.path.insert(0, '/home/atbin/workd/imputation/package/src/imputation/')
import utils
import numpy as np
import flux_config as conf

begin_date, end_date = conf.timestamps['begin_date'], conf.timestamps['end_date']

begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d %H:%M:%S")
end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")

In [13]:
df = pd.read_csv('../data_out/Calperum_L3_imputed.csv', parse_dates=['DateTime'])

df = df.loc[(df['DateTime']<=end_date) & (df['DateTime']>=begin_date)]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 42432 to 59951
Data columns (total 23 columns):
Ah                               17166 non-null float64
DateTime                         17520 non-null datetime64[ns]
ACCESS_Ah_00                     17520 non-null float64
ACCESS_Ah_01                     17520 non-null float64
ACCESS_Ah_02                     17520 non-null float64
ACCESS_Ah_10                     17520 non-null float64
ACCESS_Ah_11                     17520 non-null float64
ACCESS_Ah_12                     17520 non-null float64
ACCESS_Ah_20                     17520 non-null float64
ACCESS_Ah_21                     17520 non-null float64
ACCESS_Ah_22                     17520 non-null float64
BIOS2_Ah                         17520 non-null float64
day                              17520 non-null int64
month                            17520 non-null int64
week                             17520 non-null int64
hour                             17520 non-nul

In [15]:
ytest_not_nan = df['Ah_Calperum'].values[~df['Ah_Calperum'].isna()]
ytest_predicted_not_nan = df['Ah_predicted_Classical Linear'][~df['Ah_Calperum'].isna()]

rmse, r2, mbe = utils.diagnostic_stats(ytest_not_nan, ytest_predicted_not_nan)
std = ytest_predicted_not_nan.std()
corr = np.corrcoef(ytest_not_nan, ytest_predicted_not_nan)[0,1]

RMSE: 1.583, R^2: 0.551, MBE: -0.336


In [16]:
print(std, corr)

2.1374303177999305 0.7425291486902518


In [17]:
def taylor_diagram(self):
    summary_df = self._read_summary_file()

    ttest_min_set = summary_df['Test_begin'].unique()

    for t in ttest_min_set:
        a_summary_df = summary_df[summary_df['Test_begin']==t]
        self._make_save_taylor_diagram(a_summary_df, t)        

def _make_save_taylor_diagram(self, a_summary_df, ttest_min):

    sample = a_summary_df[['std', 'Corr.', 'Solver']].values
    refstd = 0.75*sample[:,0].min()
    srange = (0, 1.5*sample[:,0].astype(float).max()/refstd)

    ttest_min = pd.to_datetime(str(ttest_min)) 
    ttest_min =  ttest_min.strftime("%Y-%m-%d")

    title = self.primary_st + '_' +  ttest_min
    fn = '../../plot/' + title

    plot_utils.taylor_diagram(sample,  refstd, srange, title)    

    if self.conf.result['save_plots'] == True:
        plt.savefig(fn + '.jpeg')
        plt.close()

    pass


In [18]:
df = pd.read_csv('../data_out/Calperum_L3__summary_stats.csv', parse_dates=['Test_begin', 'Test_end'])


In [19]:
df

,Best_params,Corr.,MBE,R2,RMSE,Solver,Test_begin,Test_end,std
0,"{'fit_intercept': True, 'normalize': True}",0.822465,0.218771,0.676449,1.435748,Classical Linear,2013-01-01,2013-04-10,2.415209
1,"{'max_depth': 5, 'n_estimators': 500}",0.774517,0.054970,0.599876,1.580009,Random Forest,2013-01-01,2013-04-10,2.360851
2,"{'fit_intercept': True, 'normalize': True}",0.857045,0.180523,0.734526,0.980899,Classical Linear,2013-04-11,2013-07-19,1.366481
3,"{'max_depth': 5, 'n_estimators': 1000}",0.771122,0.700023,0.594629,1.382125,Random Forest,2013-04-11,2013-07-19,1.160102
4,"{'max_depth': 5, 'n_estimators': 500}",0.816733,-0.284962,0.667052,1.207553,Random Forest,2013-07-20,2013-12-31,1.469360
5,"{'fit_intercept': True, 'normalize': True}",0.715475,-0.987536,0.511905,1.924651,Classical Linear,2013-07-20,2013-12-31,2.307671
